In [1]:
import pandas as pd
import numpy as np
import itertools
from xml.etree.ElementTree import ElementTree, Element, SubElement, tostring
import datetime

In [2]:
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
def datetime_to_secs(time):  
    if type(time) is datetime.time:
        secs = time.hour * 3600 + time.minute * 60 + time.second 
        return secs
    else:
        print('Provide a datetime.time var')

In [4]:
def randNormal_time(time, dev):
    mean = datetime_to_secs(datetime.datetime.strptime(time, '%H:%M').time())
    std = datetime_to_secs(datetime.datetime.strptime(dev, '%H:%M').time())
    # Draw from a normal distr
    time_secs = np.random.normal(loc=mean, scale=std)
    rand_time = str(datetime.timedelta(seconds=time_secs)).split('.')[0]
    return rand_time

In [5]:
class Agent(object):                   
    """ Agent class """
    def __init__(self, **kwargs):
        prop_defaults = {
            'id': 'non_defined'
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))

        self.plans = list()

In [6]:
class Plan(object):
    """ Plans class """
    def __init__(self, **kwargs):
        prop_defaults = {
            'selected': 'no'
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))

        self.activities = list()

In [7]:
class Activity(object):
    """ Activities class """    
    def __init__(self, **kwargs):
        prop_defaults = {
            'type': 'undefined',
            'link': None,
            'x': None,
            'y': None,
            'end_time': None
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))
        
        self.legs = list()

In [8]:
class Leg(object):
    """ Plans class """
    def __init__(self, **kwargs):
        prop_defaults = {
            'mode': 'car'
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))

        self.route = None

In [9]:
class Route(object):
    """ Plans class """
    def __init__(self, **kwargs):
        prop_defaults = {
            'type': 'links'
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))

        self.value = None

In [10]:
def write_xml_elem2(c, d, p):
    import inspect
    
    class_name = c.__class__.__name__
    
    for attr, v in c.__dict__.items():  
        if isinstance(v, list):
            if inspect.isclass(v[0]):
                parent_elem = d[class_name]
                SubElement(parent_elem, class_name.lower())
                for e in v:
                    write_xml_elem(e, d, parent_elem)
        else:
            if v:
                xml_elem = p
                xml_elem.set(attr, str(v))

In [11]:
def write_xml_elem(c, d, p):
    import inspect
    
    class_name = c.__class__.__name__
    
    for attr, v in c.__dict__.items():  
        if isinstance(v, list):
            if inspect.isclass(v[0]):
                parent_elem = d[class_name]
                SubElement(parent_elem, class_name.lower())
                for e in v:
                    write_xml_elem(e, d, parent_elem)

In [12]:
def build_pop_xml(pop):
    
    xml_root = Element('population')
    xml_person = SubElement(root, 'person')
    xml_plan = SubElement(xml_person, 'plan')
    xml_activity = SubElement(xml_plan, 'act')
    xml_leg = SubElement(xml_plan, 'leg')
    xml_route = SubElement(xml_leg, 'route')

    parent_d = {}
    parent_d['Agent'] = xml_root
    parent_d['Plan'] = xml_person
    parent_d['Activity'] = xml_plan
    parent_d['Leg'] = xml_plan
    parent_d['Route'] = xml_leg
        
    for person in pop:
        write_xml_elem(person, parent_d, xml_root)
    return root

def build_pop_xml2(pop):
        
    root = Element('population')   
    
    d['Agent'] = xml_person
    d['Plan'] = xml_plan
    d['Activity'] = xml_activity
    d['Leg'] = xml_leg
    d['Route'] = xml_route
                    
        
    xml_person = SubElement(root, 'person')
    xml_plan = SubElement(xml_person, 'plan')
    xml_activity = SubElement(xml_plan, 'act')
    xml_leg = SubElement(xml_plan, 'leg')
    xml_route = SubElement(xml_leg, 'route')
                   
    for person in pop:
        for plan in person.plans:
            for act in plan.activities: 
                for leg in act.legs:
                    write_xml_elem(person, d)
    return root


In [14]:
#xml = build_pop_xml(pop)

# Inputs

## Zones

In [15]:
zone_centroids = pd.read_csv('../Network/Zones/zones_centroids_EPSG6312.csv', index_col=3)
zone_centroids = zone_centroids[['x', 'y']]
zone_centroids.head()

,x,y
NO,,
1,"232,596.84","393,536.79"
2,"232,441.76","393,589.67"
3,"232,129.63","393,516.97"
4,"232,045.40","393,972.93"
5,"231,761.84","393,963.87"


## Demand

In [16]:
mats = pd.read_csv('../Demand/matrices_demand.txt', delimiter=';', index_col=[0,1,2])
mats_info = pd.read_csv('../Demand/matrices_info.txt', delimiter=';', index_col=0)

C:\Users\Work\Miniconda3\envs\py3\lib\site-packages\numpy\lib\arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [17]:
df = mats.join(mats_info.NAME)
df.reset_index(inplace=True)
df = df.drop('MATRIXNO', 1)
df['NAME'] = df['NAME'].astype('category')
df = df.set_index(['NAME', 'FROMNO', 'TONO'])
df.sort_index(inplace=True)

In [18]:
demand = df.loc[['HBW_C', 'HBW_X', 'HBEDU_C', 'HBEDU_X', 'HBO_C', 'HBO_X', 'HBSH_C', 'HBSH_X', 'NHB_C', 'NHB_X',],:]

In [19]:
# Cleaning
demand.reset_index(inplace=True)
demand = demand.assign(Purpose = lambda x: x.NAME.str.split('_').str[0])
demand = demand.assign(Direction = lambda x: x.NAME.str.split('_').str[1])
demand.drop('NAME', axis=1, inplace=True)

demand.rename(columns={'FROMNO': 'From_Node', 'TONO': 'To_Node', 'VALUE': 'Trips'}, inplace=True)
demand.Purpose = demand.Purpose.astype('category')
demand.Direction = demand.Direction.astype('category')

demand.set_index(['Purpose', 'Direction', 'From_Node', 'To_Node'], inplace=True)

### Test demand

In [18]:
demand_synthPop = demand.xs(['HBW', 'C'], level=[0,1], drop_level=False)

In [19]:
# Departure time
depTime_home = '08:00' # 8 hours
depTime_home_dev = '00:30' # deviation

depTime_work = '17:00' # 8 hours
depTime_work_dev = '00:30' # deviation

In [20]:
# Create integer trips
# To-Do improve method
trips = demand_synthPop.Trips.round()

# exclude 0 trips
trips = trips[trips!=0]
# !!!!!!!!!!!!!!!!
#trips = trips.head(1)

pop = []

agent_id=1
for trip in trips.iteritems():
    idx = trip[0]
    val = trip[1]
    # Dictionary with the trips info
    d = dict(zip(trips.index.names, idx))
    for t in range(0, int(val)):
        
        # Create the agent
        new_agent = Agent(id=agent_id)
        
        # Create the plans
        # we have only one plan
        new_plan = Plan(selected='yes')        
        
        # Create the activities
        # Get Node's coords
        x, y = zone_centroids.loc[d['From_Node']]
        end_time = randNormal_time(depTime_home, depTime_home_dev)

        # To-Do must generate random points in the zone
        new_act = Activity(type='home', x=x, y=y, end_time=end_time)

        # To-Do Simplify code (functionalise)
        
        # Create the leg
        new_leg = Leg(mode='car')
        new_act.legs.append(new_leg)
        
        new_plan.activities.append(new_act)
        
        x, y = zone_centroids.loc[d['To_Node']]
        end_time = randNormal_time(depTime_work, depTime_work_dev)

        # To-Do must generate random points in the zone
        new_act = Activity(type='work', x=x, y=y, end_time=end_time)
        
        
        # Create the leg
        new_leg = Leg(mode='car')
        
        new_act.legs.append(new_leg)

        new_plan.activities.append(new_act)

        new_agent.plans.append(new_plan)
        
        pop.append(new_agent)
        agent_id+=1

In [41]:
xml = build_pop_xml(pop)

In [42]:
# Write the xml
ElementTree(xml).write('pop.xml')